Рассмотрим управляемый маятник.

Примем все константы равными единице ($g=1, L=1$). Уравнение движения математического маятника с управлением в фазовом пространстве можно записать в векторной форме.

Пусть вектор состояния $\mathbf{x}$ определяется как:
$$
\mathbf{x} = \begin{pmatrix} \theta \\ \dot{\theta} \end{pmatrix}
$$
где $\theta$ - угол отклонения маятника, а $\dot{\theta}$ - его угловая скорость.

Тогда система уравнений в форме Коши будет выглядеть так:
$$
\frac{d\mathbf{x}}{dt} = \begin{pmatrix} \dot{\theta} \\ -\sin(\theta) + u \end{pmatrix}
$$
где $u$ - управляющее воздействие (например, момент силы), с ограничением:
$$
-1 \le u \le 1
$$

Если обозначить $x_1 = \theta$ и $x_2 = \dot{\theta}$, то получим:
$$
\frac{d}{dt}\begin{pmatrix} x_1 \\ x_2 \end{pmatrix} = \begin{pmatrix} x_2 \\ -\sin(x_1) + u \end{pmatrix}
$$

Или в более компактном виде с использованием "точечной" нотации для производных по времени:
$$
\begin{cases} 
\dot{x}_1 = x_2 \\
\dot{x}_2 = -\sin(x_1) + u
\end{cases}
$$

## Математический аппарат оптимизации траекторий

Рассмотрим метод построения оптимального управления для динамической системы, описываемой уравнением маятника. Алгоритм основан на построении и анализе дерева достижимости в фазовом пространстве с последующей оптимизацией по критерию максимума площади.

### 1. Постановка задачи

Рассматривается система:
$$
\dot{\mathbf{x}} = f(\mathbf{x}, u) = \begin{pmatrix} x_2 \\ -\sin(x_1) + u \end{pmatrix}
$$
где $\mathbf{x} = (x_1, x_2)$ — вектор состояния в фазовом пространстве, а $u \in [-1, 1]$ — ограниченное управление.

### 2. Построение дерева достижимости

1.  **Начальное состояние (Корень)**: Выбирается произвольная точка в фазовом пространстве $\mathbf{x}_0$ как корень дерева.

2.  **Вершины первого уровня ("дети")**: Из корневой точки $\mathbf{x}_0$ вычисляются четыре новые вершины. Они соответствуют конечному состоянию системы после интегрирования на малый временной шаг $\Delta t$ с четырьмя комбинациями управления и направления времени:
    - $\mathbf{x}_1 = \mathbf{x}_0 + \int_0^{\Delta t} f(\mathbf{x}, u=+1) dt$
    - $\mathbf{x}_2 = \mathbf{x}_0 + \int_0^{\Delta t} f(\mathbf{x}, u=-1) dt$
    - $\mathbf{x}_3 = \mathbf{x}_0 - \int_0^{\Delta t} f(\mathbf{x}, u=+1) dt$ (интегрирование назад во времени)
    - $\mathbf{x}_4 = \mathbf{x}_0 - \int_0^{\Delta t} f(\mathbf{x}, u=-1) dt$ (интегрирование назад во времени)

3.  **Вершины второго уровня ("внуки")**: Процедура повторяется для каждой из четырех дочерних вершин. От каждой вершины первого уровня строится по две новые, используя управления, противоположные тем, что привели к их созданию. В результате получается 8 вершин второго уровня. Это обеспечивает исследование "расширяющейся" области достижимости.

### 3. Задача оптимизации

Ключевая идея метода — найти такие временные шаги $\Delta t_i$ для вершин второго уровня, чтобы их траектории "схлопывались", образуя замкнутые контуры максимальной площади.

1.  **Формирование пар**: Восемь вершин второго уровня группируются в четыре пары. Пары формируются из вершин, которые могут быть сведены в одну точку приложением противоположных управлений.

2.  **Критерий оптимальности**: Для каждой пары ищется решение задачи оптимизации:
    $$
    (\Delta t_i^*, \Delta t_j^*) = \arg\max_{\Delta t_i, \Delta t_j} \text{Area}(\mathbf{x}_i(\Delta t_i), \mathbf{x}_j(\Delta t_j))
    $$
    при условии, что траектории встречаются: $\mathbf{x}_i(\Delta t_i) = \mathbf{x}_j(\Delta t_j)$. Здесь $\text{Area}$ — это площадь в фазовом пространстве, очерченная траекториями, ведущими от общего "родителя" к точке встречи.

3.  **Результат**: Решением задачи являются оптимальные временные шаги $(\Delta t_1^*, ..., \Delta t_8^*)$ для восьми траекторий "внуков", которые максимизируют площадь захвата и обеспечивают "сшивание" траекторий.

Этот алгоритм позволяет находить локально-оптимальные управляющие последовательности, которые эффективно переводят систему между состояниями.


Таким образом получается добиться локальной области достижимости системы, причем размерности самого пространства, в данном случае 2. 

Вопросы начинаются на размерности 3 и как это вообще обобщать. Одно из следствий принципа максимума Понтрягина говорит, что для $n$-мерного пространства для построения локальной области достижимости мне понадобиться последовательность из $n$ кусочно постоянных управлений. Правда для недоуправляемых систем может оказаться больше и нужно рассматривать скобки Ли разных полей. Например дифференциальный привод. 


---

### Использование скобок Ли для базиса управления дифференциального привода

Рассмотрим их применение для классической задачи — управления мобильным роботом с дифференциальным приводом.

#### 1. Кинематическая модель системы

Состояние робота в плоскости описывается тремя переменными: его координаты `(x, y)` и угол ориентации `θ`. Вектор состояния:
$$
\mathbf{q} = \begin{pmatrix} x \\ y \\ \theta \end{pmatrix}
$$
Робот управляется двумя входами: линейной скоростью `v` и угловой скоростью `ω`. Кинематические уравнения движения выглядят следующим образом:
$$
\dot{x} = v \cos(\theta) \\
\dot{y} = v \sin(\theta) \\
\dot{\theta} = \omega
$$

#### 2. Представление в виде векторных полей

Перепишем систему в стандартной для теории управления форме $\dot{\mathbf{q}} = \sum_{i=1}^{m} g_i(\mathbf{q}) u_i$. В нашем случае $u_1 = v$ и $u_2 = \omega$.
$$
\dot{\mathbf{q}} = \begin{pmatrix} \dot{x} \\ \dot{y} \\ \dot{\theta} \end{pmatrix} = 
\begin{pmatrix} \cos(\theta) \\ \sin(\theta) \\ 0 \end{pmatrix} v + 
\begin{pmatrix} 0 \\ 0 \\ 1 \end{pmatrix} \omega
$$
Отсюда мы получаем два базовых векторных поля:
- $g_1(\mathbf{q}) = \begin{pmatrix} \cos(\theta) \\ \sin(\theta) \\ 0 \end{pmatrix}$ — соответствует движению **вперед/назад**.
- $g_2(\mathbf{q}) = \begin{pmatrix} 0 \\ 0 \\ 1 \end{pmatrix}$ — соответствует **повороту на месте**.

Очевидно, что робот не может двигаться напрямую вбок (т.е. нет вектора, который бы изменял `x` и `y` перпендикулярно текущему направлению `θ`).

#### 3. Вычисление скобки Ли

Скобка Ли двух векторных полей `f` и `g` определяется как:
$$
[f, g] = \frac{\partial g}{\partial \mathbf{q}} f - \frac{\partial f}{\partial \mathbf{q}} g
$$
Она генерирует новое векторное поле, соответствующее инфинитезимальному движению, которое можно получить, следуя сначала по `f`, затем по `g`, потом назад по `f` и назад по `g`.

Вычислим скобку Ли для наших полей $g_1$ и $g_2$:
$$
[g_1, g_2] = \frac{\partial g_2}{\partial \mathbf{q}} g_1 - \frac{\partial g_1}{\partial \mathbf{q}} g_2
$$
Сначала найдем матрицы Якоби:
$$
\frac{\partial g_1}{\partial \mathbf{q}} = \frac{\partial}{\partial (x, y, \theta)} \begin{pmatrix} \cos(\theta) \\ \sin(\theta) \\ 0 \end{pmatrix} = 
\begin{pmatrix}
0 & 0 & -\sin(\theta) \\
0 & 0 & \cos(\theta) \\
0 & 0 & 0
\end{pmatrix}
$$
$$
\frac{\partial g_2}{\partial \mathbf{q}} = \frac{\partial}{\partial (x, y, \theta)} \begin{pmatrix} 0 \\ 0 \\ 1 \end{pmatrix} = 
\begin{pmatrix}
0 & 0 & 0 \\
0 & 0 & 0 \\
0 & 0 & 0
\end{pmatrix}
$$
Теперь подставим их в формулу:
$$
[g_1, g_2] = 
\begin{pmatrix} 0 \\ 0 \\ 0 \end{pmatrix} - 
\begin{pmatrix}
0 & 0 & -\sin(\theta) \\
0 & 0 & \cos(\theta) \\
0 & 0 & 0
\end{pmatrix}
\begin{pmatrix} 0 \\ 0 \\ 1 \end{pmatrix}
= - \begin{pmatrix} -\sin(\theta) \\ \cos(\theta) \\ 0 \end{pmatrix}
= \begin{pmatrix} \sin(\theta) \\ -\cos(\theta) \\ 0 \end{pmatrix}
$$

#### 4. Формирование базиса управления

Мы получили новое векторное поле:
$$
g_3 = [g_1, g_2] = \begin{pmatrix} \sin(\theta) \\ -\cos(\theta) \\ 0 \end{pmatrix}
$$
Это векторное поле соответствует **движению вбок** (латеральному), перпендикулярно текущему направлению робота. Хотя такое движение недоступно напрямую, скобка Ли показывает, что его можно аппроксимировать комбинацией базовых движений (например, "параллельная парковка").

Теперь у нас есть три векторных поля:
- $g_1 = (\cos\theta, \sin\theta, 0)^T$ (вперед/назад)
- $g_2 = (0, 0, 1)^T$ (поворот)
- $g_3 = (\sin\theta, -\cos\theta, 0)^T$ (вбок)

Чтобы проверить, образуют ли они базис в 3D пространстве состояний, составим из них матрицу и проверим ее ранг (или найдем определитель):
$$
\text{span}\{g_1, g_2, g_3\} = \text{det}
\begin{pmatrix}
\cos(\theta) & 0 & \sin(\theta) \\
\sin(\theta) & 0 & -\cos(\theta) \\
0 & 1 & 0
\end{pmatrix}
$$
Раскладывая по второму столбцу, получаем:
$$
\text{det} = -1 \cdot (\cos(\theta)(-\cos(\theta)) - \sin(\theta)\sin(\theta)) = -1 \cdot (-\cos^2\theta - \sin^2\theta) = -1 \cdot (-1) = 1
$$
Поскольку определитель равен 1 (и не равен нулю), векторы $g_1, g_2, g_3$ линейно независимы при любом `θ`. Это означает, что они образуют базис в пространстве состояний.

**Вывод:** Система полностью управляема. Комбинируя два доступных управления (линейную и угловую скорость), мы можем переместить робота в любую точку `(x, y)` с любой ориентацией `θ`. Набор `{g₁, g₂, [g₁, g₂]}` является **базисом управления** для данной системы.

---


# Теперь вопрос

В случае с маятником получилось срастить внуков дерева и замастить часть фазовой плоскости. Для этой области можно гарантировать достижимость системы откуда-то куда-то. В трехмерном пространстве по аналогии нужно получить уже трехмерную фигуру, а значит нужно построить какие-то ребра и какие-то грани, которые в итоге сомкнуться. 

Конечная цель -- для заданной точки фазового пространства дифференциального привода получить многогранник-граф достижимости размерности 3. 